## Clustering

Consider hierarchical methods. Since these should be able to extract concepts better.

In [132]:
print('------------------------------------------------------')
print('Step 4:  K Means ')
from datetime import datetime as dt
t_start = dt.now()
print('Starting at', t_start)
print('------------------------------------------------------')

------------------------------------------------------
Step 4:  K Means 
Starting at 2018-02-16 20:55:48.071161
------------------------------------------------------


In [133]:
# Import dependencies
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from datetime import datetime as dt

In [134]:
# Configure
from config import Config as c
# set K parameter here:
n_clusters = c.n_clusters
# which cluster set to use?
# ['tsne', 'raw'] 
cluster_set = 'tsne'



# other global variables:
doi_datapath = c.dois_pkl
word_datapath = c.word_datapath
tfidf_datapath = c.tfidf_datapath
vectorizer_datapath = c.vectorizer_datapath
working_data =  c.working_data

# output paths
data_out = c.working_data
data_out_xl = c.kmeans_out_xl
data_out_csv = c.kmeans_out_csv

# load vector data
tfidf = pickle.load(open(tfidf_datapath,'rb'))
vectorizer = pickle.load(open(vectorizer_datapath,'rb'))

# global variable for ordering
dois = pickle.load(open(doi_datapath,'rb'))
data = pd.read_csv(working_data,index_col = 0)
cites = list(data.Citations)

In [135]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'this', 'before', 'among', 'only', 'any', 'eg', 'mine', 'otherwise', 'they', 'introduction', 'where', 'ltd', 'until', 'had', 'yourself', 'therefore', 'then', 'sometime', 'side', 'whereafter', 'myself', 'nine', 'even', 'etc', 'thereafter', 'will', 'name', 'some', 'front', 'mill', 'article...same', 'in', 'hereupon', 'at', 'while', 'now', 'nothing', 'beyond', 'must', 'is', 'two', 'anywhere'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

#### Clustering AFTER dimensional reduction
1. get which articles are in which cluster
2. sum the tfidf for the articles in each group
3. argsort the sum for each cluster
4. use the argsort to pick keywords from vectorizer.get_feature_names()

In [136]:
# cluster t-sne data
tsne_data = np.matrix(data[['TSNE1','TSNE2']])
tsne_kmeans = KMeans(n_clusters=n_clusters, ).fit(tsne_data) 
tsne_centers = tsne_kmeans.cluster_centers_ # centers of each cluster
tsne_klabels = tsne_kmeans.labels_ # cluster numbers for each paper

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
tsne_klabels, np.shape(tfidf)

Work out the most significant words in each cluster and add them as cluster keywords so that we know what each cluster is about.

In [ ]:
np.shape(tsne_data),np.shape(tsne_centers), tsne_kmeans

In [ ]:
dct = {}
mat = tfidf.todense() # this step kills it if dataset too large
i = 0
for label in tsne_klabels: # labels are cluster numbers
    if label in dct:
        # add together all rows with the same cluster number
        dct[label] = dct[label] + mat[i,:]
    else:
        # initialise if there's nothing to add to
        dct[label] = mat[i,:]
    i+=1
    
dct

In [ ]:
tsne_terms = vectorizer.get_feature_names() # gets the words from the vocabulary
tsne_terms_df =[]
for i in range(n_clusters):
    ls_ = []
    # get the indexes of the top 10 terms in each cluster
    indexes = list(np.array(dct[i].argsort()[:, ::-1][0])[0])[:10] # this conversion is not very neat.
#     print(indexes)
    for ind in indexes:
        ls_.append(tsne_terms[int(ind)]) 
    tsne_terms_df.append(ls_[:3])
    
# show first 3 cluster labels
tsne_terms_df[:3]

In [ ]:
# quick fix
klabels = tsne_klabels

In [ ]:
dois_ls = []
for i in range(n_clusters):
    # indices of papers that fall within ith cluster
    indices = [j for j, x in enumerate(klabels) if x ==i]
    # print(indices)
    doi_ls = [dois[j][:7] + '/' + dois[j][8:] 
             for j in indices]
    dois_ls.append(doi_ls)

# len(links_ls)

### Choose which clusters to add to your dataframe
This is not the neatest solution to this.  Do something better.

This part is a placeholder for later.  You potentially get better results by using higher dimensionality data, so KMeans on that data might be a worthwhile option.  Note that it is very slow and makes the final visualisation look scruffy.

In [ ]:
if cluster_set == 'tsne':
    terms_df = tsne_terms_df
    centers =  tsne_centers
    klabels = tsne_klabels
    kmeans = tsne_kmeans

## Find the best cited clusters

In [ ]:
## try dropping the top x percentile of papers 

nz_cites_ls = []
mean_cites_ls =[]
cites_ls_ls = []
mean_outlier_ls = []
for j in range(n_clusters):
    # get the index for each paper in jth cluster
    cluster_indices = [i for i, x in enumerate(klabels) if x ==j]
    # get the citations for those indices
    cluster_cites = [cites[i] 
                    if np.isnan(cites[i])==False 
                    else 0
                    for i in cluster_indices] # switching NaNs to zeros in order to ensure following calculations work
    
    # count non-zero cites for each cluster
    nz_cites = float(np.count_nonzero(np.array(cluster_cites)))#/len(cluster_cites))
    nz_cites_ls.append(nz_cites) # list of nz cite counts for all clusters 
    
    # exclude outliers
    # ptl80 = np.percentile(cluster_cites, 80)
#     n_outliers = 2
#     cluster_exc_max = [cites[i]
#                     if np.isnan(cites[i])==False 
#                     else 0
#                     for i in cluster_indices] # switching NaNs to zeros in order to ensure following calculations work
#     for i in range(n_outliers):
#         max_cite = max(cluster_exc_max)
#         cluster_exc_max.remove(max_cite) 
#     mean_outlier = np.mean(cluster_exc_max)
#     mean_outlier_ls.append(mean_outlier)
                     
    # work out mean citations of each cluster
    mean_cites = np.mean(np.array(cluster_cites))
    mean_cites_ls.append(mean_cites)
    
    # add the list of citations for each item in the cluster
    cites_ls_ls.append(cluster_cites)


Create a dataframe at a cluster level

In [ ]:
df = pd.DataFrame({
    'Cluster': terms_df,
    'nz_cites': nz_cites_ls,
    'mean_cites':mean_cites_ls,
    'cites':cites_ls_ls,
#     'mean_outlier':mean_outlier_ls,  
    'dois_ls':dois_ls
})

In [ ]:
df['len_cites'] = [len(i) for i in df['cites']]

In [ ]:
# df.sort_values('mean_outlier', ascending=False)

In [ ]:
df['Cluster_no'] = df.index

In [ ]:
df['nz_pc'] = df['nz_cites']/df['len_cites']

In [ ]:
r = df['nz_cites'].hist(alpha=0.3)
r

In [ ]:
# %matplotlib inline
s = df['nz_pc'].hist(alpha=0.3)
s

In [ ]:

# p = df['mean_outlier'].hist(alpha=0.3)
# q = df['mean_cites'].hist(alpha=0.3)
# p,q

In [ ]:
dois[:3]

In [ ]:
doi_cluster_dict = {}
doi_clusterno_dict = {}
for index,row in df.iterrows():
    doi_ls = row.dois_ls
    for doi in doi_ls:
        doi_cluster_dict[doi] = row.Cluster
        doi_clusterno_dict[doi] = row.Cluster_no

cluster_ls = [str(
    doi_cluster_dict[
        doi[:7]+'/'+doi[8:]
    ]) for doi in dois]

clusterno_ls = [doi_clusterno_dict[doi[:7]+'/'+doi[8:]] for doi in dois]

len(clusterno_ls),len(cluster_ls), len(dois)

In [ ]:
data

In [ ]:
article_kws_ls= []

for row in mat:
    indexes = list(np.array(row.argsort()[:, ::-1][0])[0])[:10]
    article_kws_ls.append([tsne_terms[ind] for ind in indexes][:3])

In [ ]:
article_kws_ls

In [ ]:
data['Cluster'] = cluster_ls
data['Cluster_no'] = clusterno_ls
data['Article_kws'] = article_kws_ls

In [ ]:
# data.head()

### Write data to file

In [ ]:
data.to_csv(data_out)
df.to_excel(data_out_xl)
df.to_csv(data_out_csv)

In [ ]:
s = str(dt.now()-t_start)
print('Done in '+s)